In [1]:
#!pip install scikit-learn-intelex
!pip install flaml

In [2]:
!nvidia-smi

Thu Dec  2 11:55:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 4000     Off  | 00000000:00:05.0 Off |                  N/A |
| 30%   26C    P8     5W / 125W |      0MiB /  7982MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
#from sklearnex import patch_sklearn
#patch_sklearn()

from sklearn.metrics import r2_score,accuracy_score, confusion_matrix, classification_report,precision_recall_fscore_support
import numpy as np
import pandas as pd
import sklearn
from flaml import AutoML

from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split

import xgboost as xgb

import cudf
import numpy as np
from joblib import dump, load
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns
plt.rcParams["figure.figsize"] = (20,20)

In [4]:
def create_colours(data, features):
    """Create dataframe with colour-colour data. Return an Array and list of colours.

    Args:
        data (DataFrame): Dataframe with photometric data
        features (list): List of features from photometric data
    """
    N = len(data)
    F = len(features)
    n=0
    for i in np.linspace(1,F,F,dtype=int):
    	n = n + (i-1)

    df_features = np.zeros((N, n))
    y=0
    lista=[]
    for z in np.linspace(0,F,F,dtype=int):
        for x in np.linspace(1,F-1,F-1,dtype=int):
            if z!=x and z<x:
                df_features[:,y] = np.abs(data[features[z]] - data[features[x]])
                y+=1
                lista += [features[z]+'-'+features[x]]
            else:
                pass
    df_colours = pd.DataFrame(df_features,columns = lista)
    return df_colours

def create_colours_filter(data, features):
    # Function to create colours by broadband magnitude filter
    N = len(data)
    F = len(features)
        
    df_features = np.zeros((N, F-2))
    y=0
    lista=[]
    for z in np.linspace(0,F-1,F-1,dtype=int):
        for x in np.linspace(1,F-1,F-1,dtype=int):
            if (features[z]!=features[x]) and (features[z][-2:] == features[x][-2:]):
                df_features[:,y] = np.abs(data[features[z]] - data[features[x]])
                y+=1
                lista += [features[z]+'-'+features[x]]
            else:
                pass
    df_colours = pd.DataFrame(df_features,columns = lista)
    
    return df_colours    

In [5]:
def bayes_opt_model(model,X_train, y_train, y_test):
    # model (sklearn, xgb, etc): model to optimize
    # X_train (DataFrame): Data for training
    # y_train (Series): Data to predict. IMPORTANT: Specify the variable to target
    params = {'n_estimators': [1000,2000],
          'max_depth': [0, 12],
         'learning_rate': [0.1,1]}

    opt = BayesSearchCV(
        model,
        params,
        n_iter=5,
        cv=3
    )

    opt.fit(X_train, y_train)

    print("val. score: %s" % opt.best_score_)
    print("test score: %s" % opt.score(X_test, y_test))
    
    return opt.best_estimator_

from sklearn.model_selection import RepeatedKFold

def kf_oof(clf,X_train,y_train,X_test, var,nkf):

    kf= RepeatedKFold(n_splits=5,random_state=42)
    kf.get_n_splits(features)

    oof_predictions = np.zeros(len(X_train))
    oof_test_predictions = np.zeros(len(X_test))
    i=0
    
    for train_index, holdout_index in kf.split(X_train, y_train[var]):
        i+=1

        clf.fit(X_train.iloc[train_index], y_train[var].iloc[train_index])
    
        y_pred = clf.predict(X_train.iloc[holdout_index])
        oof_predictions[holdout_index] = y_pred
    
        y_test_pred = clf.predict(X_test)
        oof_test_predictions += y_test_pred
        print("For cycle loop:",i)
        

    oof_test_predictions = oof_test_predictions/i
    return oof_predictions, oof_test_predictions

In [6]:
def df_cat_outliers(x, var1, var2):
    met = np.abs(pd.Series(x[var2]-x[var1]))
    f_out = met/(1+x[var1].astype(np.float32))
    y_outlier = pd.Series(np.where(f_out > 0.15, 'outlier', 'not outlier')).reindex(x.index)
    #df = x.drop(y_outlier[y_outlier=='outlier'].index.to_list())
    #df_nout = x.drop(y_outlier[y_outlier=='not outlier'].index.to_list())
    print("Outliers: \n", y_outlier.value_counts())
    return y_outlier

def flaml_opt(estimator, X_train, y_train):
    model = AutoML()
    
    if estimator == ['xgb']:
        
        settings = {
        "time_budget": 4200,  # total running time in seconds
        "metric": 'r2',  # primary metrics for regression can be chosen from: ['mae','mse','r2']
        "estimator_list": ['xgboost'],  # list of ML learners; we tune xgboost in this example
        "n_jobs": -1,
        "task": 'classification',  # task type    
        "log_file_name": 'clf.log'  # flaml log file
        }
    
    
    elif estimator == ['cb']:
        
        settings = {
        "time_budget": 4200,  # total running time in seconds
        "metric": 'r2',  # primary metrics for regression can be chosen from: ['mae','mse','r2']
        "estimator_list": ['xgboost'],  # list of ML learners; we tune xgboost in this example
        "n_jobs": -1,
        "tree_method":'gpu_hist',
        "task": 'classification',  # task type    
        "log_file_name": 'clf.log'  # flaml log file
        }
        
    elif estimator == ['lgb']:
        
        settings = {
        "time_budget": 4200,  # total running time in seconds
        "metric": 'r2',  # primary metrics for regression can be chosen from: ['mae','mse','r2']
        "estimator_list": ['lgbm'],  # list of ML learners; we tune xgboost in this example
        "n_jobs": -1,
        "task": 'classification',  # task type    
        "log_file_name": 'clf.log'  # flaml log file
        }
    elif estimator == ['all']:
        settings = {
        "time_budget": 4200,  # total running time in seconds
        "metric": 'r2',  # primary metrics for regression can be chosen from: ['mae','mse','r2']
        "n_jobs": -1,
        "task": 'classification',  # task type    
        "log_file_name": 'clf.log'  # flaml log file
        }
        
        
    model.fit(X_train=X_train, y_train=y_train, **settings)

    print('Best estimator:', model.best_config_per_estimator)
    return model

# Feature Engeneering

In [ ]:
# Read CSV from DR15 SDSS

df_psf = cudf.read_csv('./Data/SDSS_DR15_zspec.csv')
list_features = ['specObjID','psfMag_u', 'psfMag_g','psfMag_r','psfMag_i','psfMag_z',
                 'w1mpro','w2mpro','w3mpro','w4mpro', 'z','class']
df_psf = df_psf[list_features]
df_psf['z'] = df_psf['z']+1

# Read modelMag magnitudes
df_mag = cudf.read_csv('./Data/SDSS_DR15_mag_PCunha.csv')
list_mag = ['specObjID','modelMag_u', 'modelMag_g',
       'modelMag_r', 'modelMag_i', 'modelMag_z']
df_model = df_mag[list_mag]

# Read modelMag magnitudes
df_mag2 = cudf.read_csv('./Data/SDSS_DR15_add_photo_PCunha.csv')
list_mag2 = ['specObjID','petroMag_u', 'petroMag_g', 'petroMag_r', 'petroMag_i',
       'petroMag_z', 'cModelMag_u', 'cModelMag_g', 'cModelMag_r',
       'cModelMag_i', 'cModelMag_z']
df_model2 = df_mag2[list_mag2]

# Read 2MASS and FIRST data
#df_add = pd.read_csv('./Data/SDSS_DR15_phot_2MASS_first_PCunha.csv')
#list_add = ['specObjID','j', 'h', 'k','first_peak', 'first_integrated']
#df_add = df_add[list_add]

#df = pd.merge(df_psf, df_add, on='specObjID', how='outer')
df = cudf.merge(df_psf,df_model,on='specObjID', how='inner')
df = cudf.merge(df,df_model2,on='specObjID', how='inner')

# Read coords data
df_coords = cudf.read_csv('./Data/SDSS_DR15_coords_PCunha.csv')

df = cudf.merge(df,df_coords,on='specObjID', how='inner')

In [ ]:
df.columns

In [ ]:
del(df_psf)
del(df_model)
del(df_model2)

In [ ]:
# Drop duplicates

df = df.drop_duplicates(subset='specObjID', keep="last")

# Convert to Pandas due to colour functions
df = df.to_pandas()

In [ ]:
l_sdss2 = ['cModelMag_u', 'cModelMag_g','cModelMag_r', 'cModelMag_i', 'cModelMag_z']

l_sdss3 = ['modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i', 'modelMag_z']

l_sdss4 = ['psfMag_u', 'psfMag_g','psfMag_r','psfMag_i','psfMag_z',
         'cModelMag_u', 'cModelMag_g','cModelMag_r', 'cModelMag_i', 'cModelMag_z']

l_sdss5 = ['psfMag_u', 'psfMag_g','psfMag_r','psfMag_i','psfMag_z',
           'modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i', 'modelMag_z']

l_sdss_wise = ['psfMag_u', 'psfMag_g','psfMag_r','psfMag_i','psfMag_z',
         'w1mpro','w2mpro','w3mpro','w4mpro']


df_colours_sdss2 = create_colours(df, l_sdss2)
df_colours_sdss3 = create_colours(df, l_sdss3)
df_colours_sdss4 = create_colours_filter(df, l_sdss4)
df_colours_sdss5 = create_colours_filter(df, l_sdss5)
df_colours_sdss_wise = create_colours(df, l_sdss_wise)

In [ ]:
df = pd.concat([df.reset_index(drop=True),df_colours_sdss2.reset_index(drop=True)], axis=1, sort=False)
df = pd.concat([df.reset_index(drop=True),df_colours_sdss3.reset_index(drop=True)], axis=1, sort=False)
df = pd.concat([df.reset_index(drop=True),df_colours_sdss4.reset_index(drop=True)], axis=1, sort=False)
df = pd.concat([df.reset_index(drop=True),df_colours_sdss5.reset_index(drop=True)], axis=1, sort=False)
df = pd.concat([df.reset_index(drop=True),df_colours_sdss_wise.reset_index(drop=True)], axis=1, sort=False)

del(df_colours_sdss2)
del(df_colours_sdss3)
del(df_colours_sdss4)
del(df_colours_sdss5)
del(df_colours_sdss_wise)

In [ ]:
df['w1-w2-w3'] = np.abs(df['w1mpro']-df['w2mpro']-df['w3mpro'])
df['w1-w2-w3-w4'] = np.abs(df['w1mpro']-df['w2mpro']-df['w3mpro']-df['w4mpro'])

In [ ]:
df.columns.to_list()

In [ ]:
df['specObjID'].is_unique

# Data cleaning

In [ ]:
features = df.columns.to_list()
features.remove('class')
features.remove('z')
features.remove('specObjID')
features.remove('ra')
features.remove('dec')
for i in range(len(features)):
    print('Feature '+str(features[i])+': ',df[features[i]].loc[df[features[i]]<0].value_counts(dropna=False))
    print('Max value :',df[features[i]].max())
    print('Min value :',df[features[i]].min(),'\n')

In [ ]:
for i in range(len(features)):
    df[features[i]].loc[df[features[i]] > 50] = -9999.0

In [ ]:
for i in range(len(features)):
    print('Feature '+str(features[i])+': ',df[features[i]].loc[df[features[i]]<0].value_counts(dropna=False))
    print('Max value :',df[features[i]].max())
    print('Min value :',df[features[i]].min(),'\n')

# Data scalling

In [ ]:
# Class convertion into 0, 1, 2
le = sklearn.preprocessing.LabelEncoder()
df['class'] = le.fit_transform(df['class'])
print('Labels [0,1,2]: ',le.inverse_transform([0,1,2]))
print('Data encoded.\n')

In [ ]:
for i in range(len(features)):
    df[features[i]] = pd.to_numeric(df[features[i]], downcast="float")

In [ ]:
df.isnull().any().to_list()

In [ ]:
df

# Export data for photo_z predictions

In [ ]:
df.to_parquet("./Data/SDSS_DR15_data.parquet")

# Import photo_z predictions 

In [7]:
df = pd.read_parquet('./Data/SDSS_DR15_data.parquet')

# Add predicted photo_z

In [8]:
# Add photometric redshifts predictions
#df_meta_z2 = pd.read_csv('./Data/oof_z_spec_pred_v2.csv')
#df_meta_z2.columns=['index','meta_z2']
#df_meta_z2.set_index('index', inplace=True)
#df = pd.merge(df,df_meta_z2, left_index=True, right_index=True)

#del(df_meta_z2)

In [9]:
df_imp_z = pd.read_csv('./Data/oof_z_pred_v2.csv')
df_imp_z.columns=['index','imp_z']
df_imp_z.set_index('index', inplace=True)
df = pd.merge(df,df_imp_z, left_index=True, right_index=True)

del(df_imp_z)

# Add predicted catastrophic outlier in photo_z

In [10]:
def flag_cat_outliers(x, var1, var2):
    met = np.abs(pd.Series(x[var2]-x[var1]))
    f_out = met/(1+x[var1].astype(np.float32))
    y_outlier = pd.Series(np.where(f_out > 0.15, 'outlier', 'not outlier')).reindex(x.index)
    x['flag_outlier'] = pd.Series(np.where(f_out > 0.15, 0, 1))
    return x

In [11]:
df

,specObjID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,w1mpro,w2mpro,w3mpro,w4mpro,...,psfMag_z-w4mpro,w1mpro-w2mpro,w1mpro-w3mpro,w1mpro-w4mpro,w2mpro-w3mpro,w2mpro-w4mpro,w3mpro-w4mpro,w1-w2-w3,w1-w2-w3-w4,imp_z
0,299489676975171584,19.086210,17.417789,16.595810,16.034969,15.680770,11.856000,11.872,9.154,7.836,...,7.84477,0.016,2.702,4.020,2.718,4.036,1.318,9.170,17.006001,1.031694
1,299489677444933632,19.086210,17.417789,16.595810,16.034969,15.680770,11.856000,11.872,9.154,7.836,...,7.84477,0.016,2.702,4.020,2.718,4.036,1.318,9.170,17.006001,1.033450
2,299489951853078528,22.678499,20.570419,19.164761,18.745230,18.312450,14.574000,14.207,11.902,8.626,...,9.68645,0.367,2.672,5.948,2.305,5.581,3.276,11.535,20.160999,1.210131
3,299489952322840576,22.343210,20.316629,18.933720,18.372570,17.988461,14.226000,13.918,12.521,9.106,...,8.88246,0.308,1.705,5.120,1.397,4.812,3.415,12.213,21.319000,1.213816
4,299490226730985472,20.024401,19.601589,19.492901,19.371880,19.298290,15.102000,14.071,11.880,8.372,...,10.92629,1.031,3.222,6.730,2.191,5.699,3.508,10.849,19.221001,1.659689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3474913,11259270496166846464,23.912531,23.555460,22.746799,21.878040,20.049231,15.729000,15.002,11.273,8.584,...,11.46523,0.727,4.456,7.145,3.729,6.418,2.689,10.546,19.129999,2.044420
3474914,11259270771044753408,19.719530,19.538170,19.356890,18.952471,18.992161,15.687000,14.676,11.301,8.737,...,10.25516,1.011,4.386,6.950,3.375,5.939,2.564,10.290,19.027000,2.857131
3474915,11259271045922660352,26.007179,23.568859,22.621071,21.113041,20.064899,16.054001,15.412,11.976,7.921,...,12.14390,0.642,4.078,8.133,3.436,7.491,4.055,11.334,19.254999,1.938109
3474916,11259271320800567296,24.128031,22.385111,21.015579,20.358210,19.912359,16.104000,15.665,12.139,8.793,...,11.11936,0.439,3.965,7.311,3.526,6.872,3.346,11.700,20.493000,1.417181


In [12]:
# Flag outlier for improved photo_z
df['flag_outlier'] = df_cat_outliers(df, 'z', 'imp_z')

Outliers: 
 not outlier    3417900
outlier          57018
dtype: int64


In [13]:
le = sklearn.preprocessing.LabelEncoder()
df['flag_outlier'] = le.fit_transform(df['flag_outlier'])
print('Labels [0,1]: ',le.inverse_transform([0,1]))
print('Data encoded.\n')

Labels [0,1]:  ['not outlier' 'outlier']
Data encoded.



In [14]:
features = df.columns.to_list()
features.remove('class')
features.remove('z')
features.remove('flag_outlier')

targets = ['class','flag_outlier']

X_train, X_test, y_train, y_test = train_test_split(df[features], 
                                                    df[targets], 
                                                    test_size=0.2, 
                                                    shuffle =True, 
                                                    random_state=0,
                                                    stratify=df['flag_outlier'])


In [16]:
automl_clf = flaml_opt(['all'], X_train, y_train['flag_outlier'])

[flaml.automl: 12-02 10:08:46] {1926} INFO - task = classification
[flaml.automl: 12-02 10:08:46] {1928} INFO - Data split method: stratified
[flaml.automl: 12-02 10:08:46] {1932} INFO - Evaluation method: holdout
[flaml.automl: 12-02 10:09:03] {1999} INFO - Minimizing error metric: 1-r2
[flaml.automl: 12-02 10:09:03] {2051} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'catboost', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1']
[flaml.automl: 12-02 10:09:03] {2291} INFO - iteration 0, current learner lgbm
[flaml.automl: 12-02 10:09:04] {2405} INFO - Estimated sufficient time budget=1917556s. Estimated necessary time budget=47076s.
[flaml.automl: 12-02 10:09:04] {2485} INFO -  at 223.9s,	estimator lgbm's best error=1.0167,	best estimator lgbm's best error=1.0167
[flaml.automl: 12-02 10:09:04] {2291} INFO - iteration 1, current learner lgbm
[flaml.automl: 12-02 10:09:05] {2485} INFO -  at 224.3s,	estimator lgbm's best error=1.0167,	best estimator lgbm's best error=1.0167
[

[flaml.automl: 12-02 10:09:22] {2291} INFO - iteration 35, current learner xgboost
[flaml.automl: 12-02 10:09:23] {2485} INFO -  at 242.2s,	estimator xgboost's best error=0.9489,	best estimator xgboost's best error=0.9489
[flaml.automl: 12-02 10:09:23] {2291} INFO - iteration 36, current learner xgboost
[flaml.automl: 12-02 10:09:23] {2485} INFO -  at 242.8s,	estimator xgboost's best error=0.9489,	best estimator xgboost's best error=0.9489
[flaml.automl: 12-02 10:09:23] {2291} INFO - iteration 37, current learner xgboost
[flaml.automl: 12-02 10:09:24] {2485} INFO -  at 244.1s,	estimator xgboost's best error=0.9489,	best estimator xgboost's best error=0.9489
[flaml.automl: 12-02 10:09:24] {2291} INFO - iteration 38, current learner xgboost
[flaml.automl: 12-02 10:09:26] {2485} INFO -  at 245.3s,	estimator xgboost's best error=0.9489,	best estimator xgboost's best error=0.9489
[flaml.automl: 12-02 10:09:26] {2291} INFO - iteration 39, current learner xgboost
[flaml.automl: 12-02 10:09:27

[flaml.automl: 12-02 10:13:20] {2291} INFO - iteration 72, current learner xgb_limitdepth
[flaml.automl: 12-02 10:13:21] {2485} INFO -  at 480.4s,	estimator xgb_limitdepth's best error=0.9307,	best estimator xgboost's best error=0.6969
[flaml.automl: 12-02 10:13:21] {2291} INFO - iteration 73, current learner xgb_limitdepth
[flaml.automl: 12-02 10:13:22] {2485} INFO -  at 481.3s,	estimator xgb_limitdepth's best error=0.9307,	best estimator xgboost's best error=0.6969
[flaml.automl: 12-02 10:13:22] {2291} INFO - iteration 74, current learner xgb_limitdepth
[flaml.automl: 12-02 10:13:24] {2485} INFO -  at 483.3s,	estimator xgb_limitdepth's best error=0.9298,	best estimator xgboost's best error=0.6969
[flaml.automl: 12-02 10:13:24] {2291} INFO - iteration 75, current learner xgboost
[flaml.automl: 12-02 10:15:20] {2485} INFO -  at 600.0s,	estimator xgboost's best error=0.6969,	best estimator xgboost's best error=0.6969
[flaml.automl: 12-02 10:15:20] {2291} INFO - iteration 76, current lea

[flaml.automl: 12-02 10:33:57] {2485} INFO -  at 1716.7s,	estimator extra_tree's best error=0.9367,	best estimator xgboost's best error=0.6864
[flaml.automl: 12-02 10:33:57] {2291} INFO - iteration 110, current learner rf
[flaml.automl: 12-02 10:34:02] {2485} INFO -  at 1722.1s,	estimator rf's best error=0.9483,	best estimator xgboost's best error=0.6864
[flaml.automl: 12-02 10:34:02] {2291} INFO - iteration 111, current learner extra_tree
[flaml.automl: 12-02 10:34:52] {2485} INFO -  at 1771.7s,	estimator extra_tree's best error=0.9367,	best estimator xgboost's best error=0.6864
[flaml.automl: 12-02 10:34:52] {2291} INFO - iteration 112, current learner xgboost
[flaml.automl: 12-02 10:39:59] {2485} INFO -  at 2078.2s,	estimator xgboost's best error=0.6864,	best estimator xgboost's best error=0.6864
[flaml.automl: 12-02 10:39:59] {2291} INFO - iteration 113, current learner lgbm
[flaml.automl: 12-02 10:39:59] {2485} INFO -  at 2078.4s,	estimator lgbm's best error=1.0133,	best estimator

Best estimator: {'lgbm': {'n_estimators': 18, 'num_leaves': 25, 'min_child_samples': 10, 'learning_rate': 0.043389227182311904, 'log_max_bin': 7, 'colsample_bytree': 0.8057683870789988, 'reg_alpha': 0.0009765625, 'reg_lambda': 0.04635370841900571, 'FLAML_sample_size': 10000}, 'rf': {'n_estimators': 4, 'max_features': 0.7006670149394767, 'max_leaves': 88, 'criterion': 'entropy', 'FLAML_sample_size': 40000}, 'catboost': {'early_stopping_rounds': 11, 'learning_rate': 0.2, 'n_estimators': 74, 'FLAML_sample_size': 10000}, 'xgboost': {'n_estimators': 777, 'max_leaves': 287, 'min_child_weight': 40.1443074313098, 'learning_rate': 0.04154046124800973, 'subsample': 0.9325030188198649, 'colsample_bylevel': 0.9581891971407994, 'colsample_bytree': 0.8717346660187126, 'reg_alpha': 0.0016352264865901315, 'reg_lambda': 0.12869217985375717, 'FLAML_sample_size': 2501940}, 'extra_tree': {'n_estimators': 4, 'max_features': 1.0, 'max_leaves': 84, 'criterion': 'gini', 'FLAML_sample_size': 2501940}, 'xgb_lim

In [15]:
automl_clf = xgb.XGBClassifier(base_score=0.5, booster='gbtree',
              colsample_bylevel=0.9581891971407994, colsample_bynode=1,
              colsample_bytree=0.8717346660187126, gamma=0, gpu_id=-1,
              grow_policy='lossguide', importance_type='gain',
              interaction_constraints='', learning_rate=0.04154046124800973,
              max_delta_step=0, max_depth=0, max_leaves=287,
              min_child_weight=40.1443074313098, missing=-9999.0,
              monotone_constraints='()', n_estimators=777, n_jobs=-1,
              num_parallel_tree=1, random_state=0,
              reg_alpha=0.0016352264865901315, reg_lambda=0.12869217985375717,
              scale_pos_weight=1, subsample=0.9325030188198649,
              tree_method='gpu_hist', use_label_encoder=False,
              validate_parameters=1, verbosity=0)

In [16]:
oof_train, oof_test = kf_oof(automl_clf,X_train,y_train,X_test,'flag_outlier',nkf=5)

For cycle loop: 1
For cycle loop: 2
For cycle loop: 3
For cycle loop: 4
For cycle loop: 5
For cycle loop: 6
For cycle loop: 7
For cycle loop: 8
For cycle loop: 9
For cycle loop: 10
For cycle loop: 11
For cycle loop: 12
For cycle loop: 13
For cycle loop: 14
For cycle loop: 15
For cycle loop: 16
For cycle loop: 17
For cycle loop: 18
For cycle loop: 19
For cycle loop: 20
For cycle loop: 21
For cycle loop: 22
For cycle loop: 23
For cycle loop: 24
For cycle loop: 25
For cycle loop: 26
For cycle loop: 27
For cycle loop: 28
For cycle loop: 29
For cycle loop: 30
For cycle loop: 31
For cycle loop: 32
For cycle loop: 33
For cycle loop: 34
For cycle loop: 35
For cycle loop: 36
For cycle loop: 37
For cycle loop: 38
For cycle loop: 39
For cycle loop: 40
For cycle loop: 41
For cycle loop: 42
For cycle loop: 43
For cycle loop: 44
For cycle loop: 45
For cycle loop: 46
For cycle loop: 47
For cycle loop: 48
For cycle loop: 49
For cycle loop: 50


In [17]:
oof_train = pd.Series(oof_train, index=X_train.index)
oof_test = pd.Series(np.round(oof_test), index=X_test.index)

In [19]:
from sklearn.metrics import  confusion_matrix, classification_report

print('OOF_train: \n ', classification_report(y_train['flag_outlier'], oof_train, digits=5))
print('\n Confusion matrix: \n', confusion_matrix(y_train['flag_outlier'], oof_train))
print('OOF_test: \n ', classification_report(y_test['flag_outlier'], oof_test, digits=5))
print('\n Confusion matrix: \n', confusion_matrix(y_test['flag_outlier'], oof_test))



OOF_train: 
                precision    recall  f1-score   support

           0    0.99059   0.99788   0.99422   2734320
           1    0.77276   0.43158   0.55384     45614

    accuracy                        0.98859   2779934
   macro avg    0.88167   0.71473   0.77403   2779934
weighted avg    0.98701   0.98859   0.98700   2779934


 Confusion matrix: 
 [[2728531    5789]
 [  25928   19686]]
OOF_test: 
                precision    recall  f1-score   support

           0    0.99073   0.99813   0.99442    683580
           1    0.79740   0.44037   0.56739     11404

    accuracy                        0.98898    694984
   macro avg    0.89406   0.71925   0.78091    694984
weighted avg    0.98756   0.98898   0.98741    694984


 Confusion matrix: 
 [[682304   1276]
 [  6382   5022]]


In [20]:
oof_flag = pd.concat([oof_train, oof_test]).sort_index()
oof_flag.name = 'oof_flag_outlier'

In [21]:
df = pd.merge(df,oof_flag, left_index=True, right_index=True)

In [22]:
df['oof_flag_outlier'] = pd.to_numeric(df['oof_flag_outlier'], downcast="float")
df['flag_outlier'] = pd.to_numeric(df['flag_outlier'], downcast="float")

In [23]:
df

,specObjID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,w1mpro,w2mpro,w3mpro,w4mpro,...,w1mpro-w3mpro,w1mpro-w4mpro,w2mpro-w3mpro,w2mpro-w4mpro,w3mpro-w4mpro,w1-w2-w3,w1-w2-w3-w4,imp_z,flag_outlier,oof_flag_outlier
0,299489676975171584,19.086210,17.417789,16.595810,16.034969,15.680770,11.856000,11.872,9.154,7.836,...,2.702,4.020,2.718,4.036,1.318,9.170,17.006001,1.031694,0.0,0.0
1,299489677444933632,19.086210,17.417789,16.595810,16.034969,15.680770,11.856000,11.872,9.154,7.836,...,2.702,4.020,2.718,4.036,1.318,9.170,17.006001,1.033450,0.0,0.0
2,299489951853078528,22.678499,20.570419,19.164761,18.745230,18.312450,14.574000,14.207,11.902,8.626,...,2.672,5.948,2.305,5.581,3.276,11.535,20.160999,1.210131,0.0,0.0
3,299489952322840576,22.343210,20.316629,18.933720,18.372570,17.988461,14.226000,13.918,12.521,9.106,...,1.705,5.120,1.397,4.812,3.415,12.213,21.319000,1.213816,0.0,0.0
4,299490226730985472,20.024401,19.601589,19.492901,19.371880,19.298290,15.102000,14.071,11.880,8.372,...,3.222,6.730,2.191,5.699,3.508,10.849,19.221001,1.659689,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3474913,11259270496166846464,23.912531,23.555460,22.746799,21.878040,20.049231,15.729000,15.002,11.273,8.584,...,4.456,7.145,3.729,6.418,2.689,10.546,19.129999,2.044420,0.0,0.0
3474914,11259270771044753408,19.719530,19.538170,19.356890,18.952471,18.992161,15.687000,14.676,11.301,8.737,...,4.386,6.950,3.375,5.939,2.564,10.290,19.027000,2.857131,0.0,0.0
3474915,11259271045922660352,26.007179,23.568859,22.621071,21.113041,20.064899,16.054001,15.412,11.976,7.921,...,4.078,8.133,3.436,7.491,4.055,11.334,19.254999,1.938109,0.0,0.0
3474916,11259271320800567296,24.128031,22.385111,21.015579,20.358210,19.912359,16.104000,15.665,12.139,8.793,...,3.965,7.311,3.526,6.872,3.346,11.700,20.493000,1.417181,0.0,0.0


# Export DataFrame in pickle format

In [24]:
df.to_pickle("./Data/SDSS_DR15_final.pkl")